# "Исследование надёжности заёмщиков" <a class="tocSkip">

## Цель исследования

<div style="border: ridge black 3px; padding: 25px">
    
<b>Целью данного исследования является определение влияния семейного положения и количества детей клиента на факт погашения кредита в срок.</b><br><br>
    
Результаты исследования будут переданы в кредитный отдел банка, который сможет использовать эти данные при построении модели кредитного скоринга. <br><br>

В ходе исследования мы произведем предобработку данных, определим ключевые (из предоставленных нам) данные для решения поставленной задачи. В конце ответим на вопросы о зависимости количества детей, разных статусов семейного положения, уровней дохода и целей кредита на факт погашения кредита в срок, сформулируем общий вывод и дадим рекомендации.  
    
</div>

## Открытие данных
**Откроем файл с данными и ознакомимся с общей информацией, а так же визуально посмотрим на внутреннее заполнение таблицы.**

In [1]:
import pandas as pd
from pymystem3 import Mystem
from collections import Counter

In [2]:
data = pd.read_csv('/datasets/data.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


1. Для анализа имеем таблицу в 21 525 строк, 12 столбцов.
2. Визуально заметно, что в 2-х столбцах в одинаковом количестве не достает наблюдений о: трудовом стаже (days_employed) и ежемесячном доходе (total_income), что вполне может говорить о том, что пропуски не случайны. 
3. Типы данных по логике соответствуют содержанию столбцов. Но для дальнейших корректных манипуляций нам необходимо будет столбцы 'days_employed' и 'total_income' с дробным типом данных (float64) перевести в целочисленный тип (int64). 


In [3]:
# посмотрим на содержание первых 20 строк таблицы:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Визуально мы видим, что есть проблема с выгрузкой и отображением данных:
    
1. Данные о стаже `days_employed` выгружены со знаком "минус", а так же в некоторых строках указывается подозрительно огромное значение в днях. Нужно будет далее проверить. 
2. В колонках `days_employed` и `total_income` много знаков после запятой, переведем далее эти показатели в целочисленный тип.
3. Регистр букв нарушен в колонке `education`. Далее приведем его в порядок.

С первого взгляда мы уже заметили, что в двух столбцах таблицы есть пропуски. Выведем еще раз на экран их количество.

In [4]:
data.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Отсутствуют в равном количестве (2174 строки) данные о: количестве дней трудового стажа и ежемесячном доходе.
Проверим в одинаковых ли строках отсутствуют эти значения. Зададим условие сразу для двух столбцов и выведем сумму таких значений.

In [5]:
data[(data['days_employed'].isnull() == True) & (data['total_income'].isnull() == True)].isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Действительно. 
Одновременно в 2174 строке нет данных о количестве дней трудового стажа и ежемесячном доходе.
Поэтому можно сказать, что это не случайность и есть причины, по которым эти данные отсутствуют (например: эти люди пенсионеры или по каким-то причинам сотрудник(и) банка не запрашивал(и) эту информацию при выдаче кредита, либо сами клиенты не предоставили эту информацию банку и банк счел это возможным). 

И для более точного понимания природы данных пропусков далее определим какой тип занятости среди них преобладает больше и в одних ли пенсионерах дело.

In [6]:
# зададим одновременное выполнения условия о пропусках в обоих столбцах
# и далее методом 'value_counts()' посмотрим на кол-во удовлетворяющих нашему условию значений

nan_data = data[(data['days_employed'].isnull() == True) & (data['total_income'].isnull() == True)]
who_nan = nan_data['income_type'].value_counts()
who_nan

сотрудник          1105
компаньон           508
пенсионер           413
госслужащий         147
предприниматель       1
Name: income_type, dtype: int64

Как мы видим - дело не в пенсионерах, у обычных сотрудников так же есть пропуски в стаже и зарплате. 

<div style="border:solid black 2px; padding: 20px">
    
На данном этапе мы: <br><br>
    
- Ознакомились с предоставленной нам таблицей, оценили тип данных каждого столбца. <br>
- Визуально просмотрели содержимое, смогли заметить некоторые проблемы в загруженных данных, которые мы описали выше.<br>
- Далее нам необходимо будет скорректировать и исправить найденные ошибки, а так же определиться и выделить основные интересующие нас данные, которые смогут ответить на поставленные перед нами вопросы исследования. <br>
    
</div>

## Предобработка данных 

<a id="#start"></a>
В качестве расшифровки столбцов датафрейма у нас есть следующие исходные данные:
- *children* — количество детей в семье
- *days_employed* — общий трудовой стаж в днях
- *dob_years* — возраст клиента в годах
- *education* — уровень образования клиента
- *education_id* — идентификатор уровня образования
- *family_status* — семейное положение
- *family_status_id* — идентификатор семейного положения
- *gender* — пол клиента
- *income_type* — тип занятости
- *debt* — имел ли задолженность по возврату кредитов
- *total_income* — ежемесячный доход
- *purpose* — цель получения кредита

Некоторые названия столбцов не отражают в достаточной мере сути своего содержания для дальнейшей работы с ними, поэтому для нашего удобства переименуем некоторые из них методом `set_axis()`:
- children —> *children_numbers*
- dob_years —> *age*
- education —> *education_level*
- income_type —> *employment_type*
- debt —> *debt_case*
- total_income —> *monthly_income*

In [7]:
data.set_axis(['children_numbers', 'days_employed', 'age', 'education_level', 'education_id', 'family_status', 'family_status_id', 'gender', 'employment_type', 'debt_case', 'monthly_income', 'purpose'], axis = 'columns', inplace = True)

# проверим, что замена произошла успешно, выведем первые несколько строк на экран
data.head(3)

,children_numbers,days_employed,age,education_level,education_id,family_status,family_status_id,gender,employment_type,debt_case,monthly_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья


In [8]:
# далее приведем в порядок регистр букв в колонке 'education_level'
data['education_level'] = data['education_level'].str.lower() 

# избавимся от знаков минус в колонке 'days_employed'
data.loc[data['days_employed'] < 0, 'days_employed'] = data['days_employed'] * (-1)

Поскольку для поставленного перед нами исследования дни стажа не помогут определить ответ на поставленные вопросы, мы просто проверим на будущее верно ли выружаются сами дни стажа (т.к. визуально мы заметили огромные значения, что похоже на сбой).

In [9]:
# Допустим, что максимально возможное кол-во лет стажа для человека = 42. Среднее количество рабочих дней в году = 247,
# выведем на экран сколько строк найдется по данному условию:

employed_artifacts = data[data['days_employed'] > (42*247)]
employed_artifacts.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3670 entries, 4 to 21521
Data columns (total 12 columns):
children_numbers    3670 non-null int64
days_employed       3670 non-null float64
age                 3670 non-null int64
education_level     3670 non-null object
education_id        3670 non-null int64
family_status       3670 non-null object
family_status_id    3670 non-null int64
gender              3670 non-null object
employment_type     3670 non-null object
debt_case           3670 non-null int64
monthly_income      3670 non-null float64
purpose             3670 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 372.7+ KB


Итого = 3670 строк. Судя по всему есть техническая проблема. Все рекомендации для поставщиков данных мы дадим в конце документа.

<div style="border:solid black 2px; padding: 20px">
На данном этапе мы:<br><br>
    
- Произвели замену некоторых наименований столбцов в сторону наиболее понятных для восприятия.<br>
- Привели в порядок регистр букв в колонке `education_level`. <br>
- Убрали лишние выгруженные знаки "-" в колонке `days_employed`. <br>
- Обнаружили проблему технического характера при выгрузке данных в колонку `days_employed. <br>
    
</div>

### Обработка пропусков

**Ежемесячный доход (в отличии от стажа) играет важную роль в нашем анализе, поэтому произведем замену пропусков:**
- Поскольку это не категорийные значения, а количественные и у нас есть данные о доходах остальных клиентов, будет логично заполнить далее эти пропуски медианой вне зависимости от трудового стажа, либо типа занятости (т.к. эти данные весьма субъективны, специфичны по представленной категоризации и не являются ориентирами для нас при заполнении пропусков, в отличии, например, от категоризации по профессиям).
- Среднее арифмитическое значение дохода не будем использовать категорически в данном случае, чтобы избежать перекоса в итоговых значениях.

In [10]:
# произведем замену пропущенных значений медианой в обоих столбцах 'days_employed' и 'monthly_income'
data['days_employed'] = data['days_employed'].fillna(data['days_employed'].median())
data['monthly_income'] = data['monthly_income'].fillna(data['monthly_income'].median())

# проверим, что пропусков не осталось
data.isnull().sum()

children_numbers    0
days_employed       0
age                 0
education_level     0
education_id        0
family_status       0
family_status_id    0
gender              0
employment_type     0
debt_case           0
monthly_income      0
purpose             0
dtype: int64

In [11]:
# далее проверим некоторые столбцы на уникальные значения и их корректность

print('Уникальные значения количества детей: ', data['children_numbers'].unique())
print('Уникальные значения семейного положения: ', data['family_status'].unique())
print('Уникальные значения пола: ', data['gender'].unique())
print('Уникальные значения случаев задолженности: ', data['debt_case'].unique())
print('Уникальные значения возраста: ', data['age'].unique())

Уникальные значения количества детей:  [ 1  0  3  2 -1  4 20  5]
Уникальные значения семейного положения:  ['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'Не женат / не замужем']
Уникальные значения пола:  ['F' 'M' 'XNA']
Уникальные значения случаев задолженности:  [0 1]
Уникальные значения возраста:  [42 36 33 32 53 27 43 50 35 41 40 65 54 56 26 48 24 21 57 67 28 63 62 47
 34 68 25 31 30 20 49 37 45 61 64 44 52 46 23 38 39 51  0 59 29 60 55 58
 71 22 73 66 69 19 72 70 74 75]


1. Мы видим некоторые неточности выгрузки. Количество детей "-1" и "20". Возможно у кого-то действительно есть 20 детей, но тут скорее всего ошибка ввода: заменим "-1" на "1", "20 на "2".
2. В значении пола судя по всему так же случайный пропуск - "XNA". Посмотрим сколько таких значений всего в таблице.
3. "Нулевые" значения возраста заменим средним значением, судя по всему это случайные пропуски.

In [12]:
# меняем значения "-1" на "1", "20 на "2" в колонке 'children_numbers'
data.loc[data['children_numbers'] == -1, 'children_numbers'] = 1
data.loc[data['children_numbers'] == 20, 'children_numbers'] = 2

# посмотрим сколько всего значений 'XNA' содержит столбец 'gender'
data['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

Значение 'XNA' в 1 экземпляре.

In [13]:
# заменим ошибочное значение 'XNA' на любое значение, например, на "F" (т.к. это единичная замена и это не критично)
data.loc[data['gender'] == 'XNA', 'gender'] = 'F'

# "нулевые" значения возраста заменим на среднее арифмитическое значение
data.loc[data['age'] == 0, 'age'] = data['age'].mean()

<div style="border:solid black 2px; padding: 20px">
    
На данном этапе мы привели в порядок содержание всех ячеек таблицы:<br><br>
    
- заполнили пропуски в столбцах `days_employed` и `monthly_income` медианой на основании вышеизложенного. <br>
- проверили важные для нашего исследования значения столбцов на корректность отображения, нашли неточности и скорректировали их.<br> 
                                                                                                            
</div>

### Замена типа данных

**Далее приведем в порядок тип данных. Как ранее упоминали нам нужно отбросить лишние знаки после запятой, а так же избежать возможных ошибок в выполнении кода: преобразуем тип данных колонок 'days_employed', 'monthly_income', а так же 'age'* (*после замены нулевых значений на среднее - столбец 'age' так же стал типа float64) в целочисленный методом 'astype()'.**


In [14]:
# выведем информацию для наглядности
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children_numbers    21525 non-null int64
days_employed       21525 non-null float64
age                 21525 non-null float64
education_level     21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
employment_type     21525 non-null object
debt_case           21525 non-null int64
monthly_income      21525 non-null float64
purpose             21525 non-null object
dtypes: float64(3), int64(4), object(5)
memory usage: 2.0+ MB


In [15]:
# заменим тип данных на 'int' 
data['days_employed'] = data['days_employed'].astype('int')
data['monthly_income'] = data['monthly_income'].astype('int')
data['age'] = data['age'].astype('int')

# проверим
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children_numbers    21525 non-null int64
days_employed       21525 non-null int64
age                 21525 non-null int64
education_level     21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
employment_type     21525 non-null object
debt_case           21525 non-null int64
monthly_income      21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


<div style="border:solid black 2px; padding: 20px">
    
Мы произвели замену типа данных колонок `days_employed` , `monthly_income` и `age` из плавающего типа в целочисленный тип методом 'astype()'.
    
Методом 'to_numeric()' не пользовались в данном случае, т.к. он преобразовывает тип данных в float, который как раз изначально мы имели.
    
</div>

### Обработка дубликатов

**Предобработку мы почти завершили, теперь остается найти и посчитать количество грубых дубликатов в нашей таблице.**
Т.к. мы привели в единый регистр столбец `education_level` теперь поиск дубликатов будет эффективным.

In [16]:
# используем метод 'duplicated()' и следом метод 'sum()' для подсчета количества
print('Дубликатов найдено:', data.duplicated().sum())

Дубликатов найдено: 71


Поскольку грубых дубликатов не так много - можно предположить, что имеет место ситуация задвоения ввода в момент сбора информации в банке, или задвоения при выгрузке таблицы.

In [17]:
# удалим эти дубликаты, одновременно сохраняя индекс каждой строки, используя метод 'reset_index(drop=True)'
data = data.drop_duplicates().reset_index(drop=True)
print('Дубликатов осталось:', data.duplicated().sum())

Дубликатов осталось: 0


<div style="border:solid black 2px; padding: 20px">
    
На данном этапе мы выполнили полную предобработку данных, обнаружили грубые дубликаты и удалили их.<br><br>
    
Можно переходить непосредственно к лемматизации и категоризации.
    
</div>

### Лемматизация

**Перед нами стоит задача найти ответ как разные цели кредита влияют на возврат кредита в срок, поэтому для начала нам нужно лемматизировать значения последнего столбца таблицы 'purpose' и найти самые распространенные цели. И затем их категоризировать по целям.**

In [18]:
# в переменную 'purpose_unique_list' сохраним уникальные значения из столбца 'purpose', используя метод 'unique()'
# далее в цикле лемматизируем каждое значение из 'purpose_unique_list' и передадим его в новый список лемм 'purpose_lemmas'
# после чего выведем на экран полученный результат:

m = Mystem()
purpose_unique_list = data['purpose'].unique()
purpose_lemmas = []
for i in purpose_unique_list:
    lemma = m.lemmatize(i)
    purpose_lemmas.append(lemma)
display(purpose_lemmas)

[['покупка', ' ', 'жилье', '\n'],
 ['приобретение', ' ', 'автомобиль', '\n'],
 ['дополнительный', ' ', 'образование', '\n'],
 ['сыграть', ' ', 'свадьба', '\n'],
 ['операция', ' ', 'с', ' ', 'жилье', '\n'],
 ['образование', '\n'],
 ['на', ' ', 'проведение', ' ', 'свадьба', '\n'],
 ['покупка', ' ', 'жилье', ' ', 'для', ' ', 'семья', '\n'],
 ['покупка', ' ', 'недвижимость', '\n'],
 ['покупка', ' ', 'коммерческий', ' ', 'недвижимость', '\n'],
 ['покупка', ' ', 'жилой', ' ', 'недвижимость', '\n'],
 ['строительство', ' ', 'собственный', ' ', 'недвижимость', '\n'],
 ['недвижимость', '\n'],
 ['строительство', ' ', 'недвижимость', '\n'],
 ['на', ' ', 'покупка', ' ', 'подержать', ' ', 'автомобиль', '\n'],
 ['на', ' ', 'покупка', ' ', 'свой', ' ', 'автомобиль', '\n'],
 ['операция', ' ', 'с', ' ', 'коммерческий', ' ', 'недвижимость', '\n'],
 ['строительство', ' ', 'жилой', ' ', 'недвижимость', '\n'],
 ['жилье', '\n'],
 ['операция', ' ', 'со', ' ', 'свой', ' ', 'недвижимость', '\n'],
 ['автомобиль'

<div style="border:solid black 2px; padding: 20px">
    
На данном этапе мы выполнили лемматизацию целей кредита по колонке `purpose`. Это существенно облегчит нам задачу в понимании основных целей кредита и их последующую категоризацию.
    
</div>

### Категоризация данных

**Произведем категоризацию целей кредита по результатам проведенной лемматизации.**
Просмотрев полученный список лемм (отбросив не существенные слова, относящиеся к действиям и прилагательным) - мы можем выделить 4 основные категории целей кредита по ключевым словам:
1. Недвижимость (или жилье)
2. Свадьба
3. Автомобиль
4. Образование

Добавим в таблицу новую колонку с этими категорию, назовем ее 'purpose_main'.

In [19]:
# для категоризации создадим функцию 'purpose_category()', которая будет проверять на вход значения из столбца 'purpose',
# и в зависимости от соответствия лемме 'pur_lemma' того же столбца функция будет возвращать ту категорию из 4-х, которой
# соответствует по заданному условию

def purpose_category(purpose):
    pur_lemma = m.lemmatize(purpose)
    if 'недвижимость' in pur_lemma or 'жилье' in pur_lemma:
        return 'недвижимость'
    elif 'свадьба' in pur_lemma:
        return 'свадьба'
    elif 'автомобиль' in pur_lemma:
        return 'автомобиль'
    elif 'образование' in pur_lemma:
        return 'образование'
    else:
        return 'другое'
    
# методом 'apply' применим созданную функцию к столбцу 'purpose' и полученные значения 
# будут переданы в новый столбец 'purpose_main'
data['purpose_main'] = data['purpose'].apply(purpose_category)

# проверим наглядно результат
data.head()

,children_numbers,days_employed,age,education_level,education_id,family_status,family_status_id,gender,employment_type,debt_case,monthly_income,purpose,purpose_main
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба


Категоризацию по целям кредита мы провели, проведем теперь **категоризацию по уровню дохода**, т.к. далее эти показатели нам будут нужны.
Создадим 4 категории с названиями уровней доходов, которые на наш взгляд адекватны и актуальны в текущих реалиях как по названию, так и по содержанию их граничащих значений:
- **Низкий** (уровень дохода менее 50 000 р.)
- **Средний** (от 50 001 р. до 100 000 р.)
- **Выше среднего** (от 100 001 р. до 200 000 руб.)
- **Высокий** (от 200 001 руб. и более)

Так же как и с категоризацией целей кредита, создадим функцию, которая поможет создать нам новый столбец с названиями категорий в соответствии от уровня дохода.

In [20]:
# создаем функцию income_category(), которая будет проверять на вход значения из столбца 'monthly_income',
# и в зависимости от выполнения подходящего условия будет возвращать название 1-й из 4-х категорий дохода

def income_category(monthly_income):
    if monthly_income <= 50000:
        return 'низкий'
    elif 50001 <= monthly_income <= 100000:
        return 'средний'
    elif 100001 <= monthly_income <= 200000:
        return 'выше среднего'
    elif monthly_income >= 200001:
        return 'высокий'
    
# методом 'apply' применим созданную функцию к столбцу 'monthly_income' и полученные значения 
# будут занесены в новый столбец таблицы 'income_level'
data['income_level'] = data['monthly_income'].apply(income_category)

# проверим результат
data.head()

,children_numbers,days_employed,age,education_level,education_id,family_status,family_status_id,gender,employment_type,debt_case,monthly_income,purpose,purpose_main,income_level
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость,высокий
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,выше среднего
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость,выше среднего
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,высокий
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,выше среднего


<div style="border:solid black 2px; padding: 20px">
    
На данном этапе мы произвели нужные нам для следующего этапа виды категоризации:<br><br>
    
- по целям кредита <br>
- по уровню дохода <br><br>
    
Категоризацию по кол-ву детей производить не было необходимости, т.к. у нас всего 6 возможных вариантов количества детей - 0, 1, 2, 3, 4, 5 - уже своего рода категории. Тоже самое относится и к семейным статусам, категоризация уже имеет место.<br>
    
</div>

## Ответы на вопросы исследования

Для ответа на вопросы исследования нам необходимо определить **средний показатель случаев задолженности** относительно той группы/категории, которую мы будем рассматривать. А так же для наглядности в дополнение создадим общую **таблицу-рейтинг** с количеством должников и добросовестных клиентов и с %-долей от общего количества всех исследуемых.

In [21]:
# проверим для начала какие значения есть в столбце о наличии задолженности:
data['debt_case'].value_counts()

0    19713
1     1741
Name: debt_case, dtype: int64

Столбец `debt_case` заполнен 0 и 1: сигнализирует либо об отсутствии случаев задолженности (0), либо наоборот о нарушении сроков выплат (1). Для составления **рейтинга о должниках** удобно будет суммировать эти (1) в столбце `debt_case`.
**Для добросовестных плательщиков** для наглядности мы создадим новый столбец также заполненный (1) при помощи функции: она проверит (0) на вход и, если клиент добросовестно выплачивает кредиты - создаст новый столбец-индикатор `responsible_rating`, заполненный единицами.

In [22]:
# создадим функцию 'no_debts', которая создаст новый столбец-индикатор 'responsible_rating', заполненный единицами
# на вход проверяется столбец 'debt_case' и при наличии 0, функция возвращает 1 
def no_debts(debt_case):
    if debt_case == 0:
        return 1
    else:
        return 0
    
# методом 'apply' применим созданную функцию к столбцу 'debt_case' и возвращенные значения будут занесены в
# столбец 'responsible_rating' 
data['responsible_rating'] = data['debt_case'].apply(no_debts)
data.head(2)

,children_numbers,days_employed,age,education_level,education_id,family_status,family_status_id,gender,employment_type,debt_case,monthly_income,purpose,purpose_main,income_level,responsible_rating
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость,высокий,1
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,выше среднего,1


**Для подсчета среднего показателя случаев задолженности** среди выбранной категории (аргумент 'category') создадим функцию  `dependence(category)`. Группируем по категории и подсчитываем среднее значение случаев задолженности методом `mean()`.

In [23]:
def dependence(category):
    return data.groupby(category)['debt_case'].mean().sort_values(ascending = False)

А так же создадим функцию `rating_summary(subject)` , которая в зависимости от выбранной исследуемой категории `category` составит нам таблицу-рейтинг с общим количеством следующих параметров: случаев просрочки `debt_case`, случаев добросовестных своевременных платежей `responsible_rating` и долей в '%' от общего числа всех исследуемых (21454 клиентов).

In [24]:
# объявляем функцию, группируем по нужной нам категории 'category' и в метод agg() передадим словари с ключами 
# 'debt_case' и 'responsible_rating', которые будем суммировать
def rating_summary(category):
    summary1 = data.groupby(category).agg({'debt_case':'sum'})
    summary1['total_clients_%'] = (summary1['debt_case'] / 21454) * 100
    summary2 = data.groupby(category).agg({'responsible_rating':'sum'})
    summary2['total_clients_%'] = (summary2['responsible_rating'] / 21454) * 100
    
# методом 'merge()' объединяем обе таблицы 'summary1' и 'summary2'
    total_summary = summary1.merge(summary2, on=category, how='left')
    return total_summary.sort_values(by = 'responsible_rating', ascending = False)

### Зависимость между наличием детей и возвратом кредита в срок

In [25]:
rating_summary('children_numbers')

,debt_case,total_clients_%_x,responsible_rating,total_clients_%_y
children_numbers,,,,
0,1063,4.954787,13028,60.725273
1,445,2.074205,4410,20.555607
2,202,0.941549,1926,8.977347
3,27,0.125851,303,1.412324
4,4,0.018645,37,0.172462
5,0,0.000000,9,0.041950


In [26]:
dependence('children_numbers')

children_numbers
4    0.097561
2    0.094925
1    0.091658
3    0.081818
0    0.075438
5    0.000000
Name: debt_case, dtype: float64

<div style="border:solid black 2px; padding: 20px">
    
Зависимость от количества детей следующая:<br><br>

1. Чем больше детей у клиента, тем сильнее риск невыплаты (не своевременной выплаты) кредита.<br>
2. Клиенты, имеющие 4 детей сильнее всего подвержены риску невыплаты (не своевременной выплаты) кредита.<br>
3. Клиенты, не имеющие детей - менее всего подвержены риску невыплаты (не своевременной выплаты) кредита.<br><br>
    
Так же стоит отметить, что клиенты имеющие 2 или более детей - берут кредиты реже, чем клиенты не имеющие детей или имеющие 1 ребенка.<br>
    
</div>

### Зависимость между семейным положением и возвратом кредита в срок

In [27]:
rating_summary('family_status')

,debt_case,total_clients_%_x,responsible_rating,total_clients_%_y
family_status,,,,
женат / замужем,931,4.339517,11408,53.174233
гражданский брак,388,1.808521,3763,17.539853
Не женат / не замужем,274,1.277151,2536,11.820640
в разводе,85,0.396197,1110,5.173860
вдовец / вдова,63,0.293652,896,4.176377


In [28]:
dependence('family_status')

family_status
Не женат / не замужем    0.097509
гражданский брак         0.093471
женат / замужем          0.075452
в разводе                0.071130
вдовец / вдова           0.065693
Name: debt_case, dtype: float64

<div style="border:solid black 2px; padding: 20px">
    
Зависимость от семейного статуса следующая:<br><br>

1. Клиенты, которые "не женаты/ не замужем", а так же те, кто проживает совместно, но не оформил отношения официально ("гражданский брак") - сильнее всего подвержены риску невыплаты (не своевременной выплаты) кредита.<br>
2. Клиенты, которые в статусе "в разводе" или "вдовец/ вдова" - менее всего подвержены риску невыплаты (не своевременной выплаты) кредита.<br>
    
Стоит отметить, что клиенты, которые состоят либо в официальных отношениях "женат/ замужем", либо в "гражданском браке" чаще остальных берут кредиты.<br>
И меньше остальных берут кредиты клиенты в статусе "в разводе" или "вдовец/ вдова".<br>
    
</div>

### Зависимость между уровнем дохода и возвратом кредита в срок


In [29]:
rating_summary('income_level')

,debt_case,total_clients_%_x,responsible_rating,total_clients_%_y
income_level,,,,
выше среднего,1029,4.796308,10896,50.787732
высокий,358,1.668686,4708,21.944626
средний,331,1.542836,3760,17.525869
низкий,23,0.107206,349,1.626736


In [30]:
dependence('income_level')

income_level
выше среднего    0.086289
средний          0.080909
высокий          0.070667
низкий           0.061828
Name: debt_case, dtype: float64

<div style="border:solid black 2px; padding: 20px">
    
Зависимость от уровня дохода следующая:<br><br>

1. Клиенты с ежемесячным доходом "выше среднего" (от 100 001 р. до 200 000 руб.) сильнее остальных подвержены  риску невыплаты (не своевременной выплаты) кредита. Но вместе с тем - это та категория, которая больше остальных представлена в заемщиках.<br>
2. Клиенты с "низким" ежемесячным уровнем дохода (менее 50 000 р.) подвержены риску меньше всех. Здесь так же стоит добавить, что таких клиентов всего 1.7% от общего числа исследуемых клиентов. И поэтому можно сказать, что это клиенты ответственные, хорошо оценивающие свои силы в условиях низкого дохода.<br>
    
</div>

### Влияние целей кредита на его возврат в срок

In [31]:
rating_summary('purpose_main')

,debt_case,total_clients_%_x,responsible_rating,total_clients_%_y
purpose_main,,,,
недвижимость,782,3.645008,10029,46.746527
автомобиль,403,1.878438,3903,18.192412
образование,370,1.724620,3643,16.980516
свадьба,186,0.866971,2138,9.965508


In [32]:
dependence('purpose_main')

purpose_main
автомобиль      0.093590
образование     0.092200
свадьба         0.080034
недвижимость    0.072334
Name: debt_case, dtype: float64

<div style="border:solid black 2px; padding: 20px">
    
Зависимость от целей кредита следующая:<br><br>
    
1. Клиенты, цели которых связаны с "автомобилем" сильнее остальных подвержены  риску невыплаты (не своевременной выплаты) кредита. Следом с похожим показателем можно выделить категорию целей "образование".<br>
2. Клиенты, цели которых связаны с "недвижимостью" подвержены риску невыплат менее всех. Так же стоит отметить, что 50.3% всех исследуемых берут кредит именно под "недвижимость" и это дает возможность утверждать, что категорию целей кредита "недвижимость" банк может выделить как "безопасную".<br>
    
</div>

## Общий вывод

<div style="border: ridge black 3px; padding: 25px">
    
<b>Выявлены самые "безопасные" категории клиентов, менее предрасположенные к невыплатам:</b><br><br>
    
1. Клиенты в статусе "женат/замужем".<br>
2. Клиенты без детей.<br>
3. Клиенты с доходом менее 50 000 руб.<br>
4. Клиенты, берущие кредит на "недвижимость".<br><br>
    
<b>"Небезопасные" категории клиентов, на которые следует обратить внимание:</b><br><br>
    
1. Клиенты  не имеющие отношений или официально оформленных отношений со статусами: "не женат/не замужем" или "гражданский брак".<br>
2. Клиенты с 2 или 4 детьми.<br>
3. Клиенты с ежемесячным доходом "выше среднего" (от 100 001 р. до 200 000 руб.).<br>
4. Клиенты, берущие кредит на "автомобиль" или "образование".<br>
    
</div>

### Рекомендации для технических специалистов

<div style="border: ridge black 2px; padding: 25px">
    
<b>В ходе предобработки данных были выявлены неточности выгрузки, исходя из которых мы рекомендуем поставщикам данных/разработчикам банка обратить внимание на следующие моменты:</b><br><br>
    
1. Привести в единый формат заполнение колонки об 'уровне образования' (обнаружен разный регистр, скорее всего сейчас заполнение происходит вручную).<br>
2. Привести в единый формат заполнение колонок 'количество детей' и 'возраст' (скорее всего сейчас заполнение происходит вручную).<br>
3. Проверить текущую систему сбора информации от клиентов о 'стаже работы' и 'ежемесячной зарплате' как обязательную при первоначальном сборе (были обнаружены одновременные пропуски в обоих колонках). На данном этапе похоже, что регламент сбора информации позволяет пропускать эти значения сотрудникам банка, либо необходимо проверить систему выгрузки данных на ошибки.<br>
4. Проверить корректность ввода/выгрузки значений в колонке о 'количестве дней стажа' (было обнаружено 3670 строк с огромными, несоответствующими реальности значениями), а так же обратить внимание на выгружающиеся знаки 'минус' в начале значений и найти причину.<br>
    
</div>